## Реализация user_based подхода

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import datetime as dt
!pip install implicit
from implicit._nearest_neighbours import all_pairs_knn
from sklearn.preprocessing import normalize

     |████████████████████████████████| 1.1MB 3.5MB/s 
  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3476930 sha256=e74790835ea875065836603aea6808a462da9b8395aea554e9a8a5312292a357
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


## обработка для считывания файлов с гугл диска

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
cd /content/gdrive/My Drive/python

/content/gdrive/My Drive/python


## Считывание и предобработка данных о действиях, тестовых и тренировочных данных

In [0]:
#функция для переиндексации столбцов клиентов и товаров
def newIndex(user_actions):
    clients = user_actions['clientid'].unique() # все уникальные айди клиентов из user_actions
    clients_cat = np.arange(0, len(clients), dtype='uint32') # массив с элементами от 0 до n - 1

    # таблица, где каждая строка переводит clientid в номер строки
    user_mapping = pd.DataFrame({'old': clients, 'new': clients_cat})

    items = user_actions['itemid'].unique() # все уникальные айди товаров из user_actions
    items_cat = np.arange(0, len(items), dtype='uint32') # массив с элементами от 0 до m - 1

    # таблица, где каждая строка переводит itemid в номер столбца
    item_mapping = pd.DataFrame({'old': items, 'new': items_cat})

    user_actions['clientid'] = user_actions['clientid'].map(user_mapping.set_index('old').new)
    user_actions['itemid'] = user_actions['itemid'].map(item_mapping.set_index('old').new)
    return (user_actions, user_mapping, item_mapping)

In [5]:
user_actions = pd.read_parquet('data/user_actions_august.parquet')
user_actions.head()

,clientid,itemid,action_type,timestamp
0,179153,135391270,view,2019-08-06 15:14:45
1,179153,135391272,view,2019-08-06 15:16:29
2,179153,135391270,view,2019-08-08 09:07:29
3,179153,142132354,view,2019-08-20 17:59:58
4,179153,140487634,view,2019-08-23 16:46:39


In [0]:
user_actions['clientid'] = user_actions['clientid'].astype('uint32')
user_actions['itemid'] = user_actions['itemid'].astype('uint32')
user_actions['action_type'] = user_actions['action_type'].map({'view': 0, 'to_cart': 1}).astype('bool')

In [7]:
#деление по просмотрамм
new_user_actions_view = user_actions[~user_actions['action_type']].reset_index(drop=True)
new_user_actions_view = new_user_actions_view.drop(columns=['timestamp', 'action_type'])
user_actions_view, user_mapping, item_mapping = newIndex(new_user_actions_view)
user_actions_view.head()

,clientid,itemid
0,0,0
1,0,1
2,0,0
3,0,2
4,0,3


In [8]:
#деление по добавлению в корзину
user_actions_tocart = user_actions[user_actions['action_type']].reset_index(drop=True)
user_actions_tocart = user_actions_tocart.drop(columns=['timestamp', 'action_type'])
user_actions_tocart,user_mapping_tocart, item_mapping_tocart = newIndex(user_actions_tocart)
user_actions_tocart.head()

,clientid,itemid
0,0,0
1,0,0
2,0,1
3,0,2
4,0,3


## Для тренировочной выборки

делим также по просмотрам и добавлениям в корзину

In [11]:
train_target = pd.read_csv('data/train_with_scores.csv')
train_target['clientid'] = train_target['clientid'].astype('uint32')
train_target['jointitemid'] = train_target['jointitemid'].astype('uint32')
train_target['label']=train_target['label'].astype('int8')
train_target['novelty_cnt']=train_target['novelty_cnt'].astype('int32')
train_target['count_on_session_view']=train_target['count_on_session_view'].astype('int32')
train_target['count_on_session_to_cart']=train_target['count_on_session_to_cart'].astype('int32')


#по просмотрам
ib_train_target_view = train_target[['clientid','itemid','jointitemid']].copy()
ib_train_target_view['user_cat'] = ib_train_target_view['clientid'].map(user_mapping.set_index('old').new)
ib_train_target_view['jointitem_cat'] = ib_train_target_view['jointitemid'].map(item_mapping.set_index('old').new)

#по картэдам
ib_train_target_tocart = train_target[['clientid','itemid','jointitemid']].copy()
ib_train_target_tocart['user_cat'] = ib_train_target_tocart['clientid'].map(user_mapping_tocart.set_index('old').new)
ib_train_target_tocart['jointitem_cat'] = ib_train_target_tocart['jointitemid'].map(item_mapping_tocart.set_index('old').new)
ib_train_target_tocart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1390438 entries, 0 to 1390437
Data columns (total 5 columns):
clientid         1390438 non-null uint32
itemid           1390438 non-null int64
jointitemid      1390438 non-null uint32
user_cat         1057532 non-null float64
jointitem_cat    1082872 non-null float64
dtypes: float64(2), int64(1), uint32(2)
memory usage: 42.4 MB


## Для тестовой выборки

делим также по просмотрам и добавлениям в корзину

In [13]:
test_target = pd.read_csv('data/test_with_scores.csv')

test_target ['clientid'] = test_target ['clientid'].astype('uint32')
test_target ['jointitemid'] = test_target ['jointitemid'].astype('uint32')
test_target ['label'] = test_target ['label'].astype('int8')
test_target ['novelty_cnt'] = test_target ['novelty_cnt'].astype('int32')
test_target ['count_on_session_view'] = test_target['count_on_session_view'].astype('int32')
test_target ['count_on_session_to_cart'] = test_target ['count_on_session_to_cart'].astype('int32')

#по проссмотрам
ib_test_target_view = test_target[['clientid','itemid','jointitemid']].copy()
ib_test_target_view['user_cat'] = ib_test_target_view['clientid'].map(user_mapping.set_index('old').new)
ib_test_target_view['jointitem_cat'] = ib_test_target_view['jointitemid'].map(item_mapping.set_index('old').new)

#картедам
ib_test_target_tocart = test_target[['clientid','itemid','jointitemid']].copy()
ib_test_target_tocart['user_cat'] = ib_test_target_tocart['clientid'].map(user_mapping_tocart.set_index('old').new)
ib_test_target_tocart['jointitem_cat'] = ib_test_target_tocart['jointitemid'].map(item_mapping_tocart.set_index('old').new)
ib_test_target_tocart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597158 entries, 0 to 597157
Data columns (total 5 columns):
clientid         597158 non-null uint32
itemid           597158 non-null int64
jointitemid      597158 non-null uint32
user_cat         456344 non-null float64
jointitem_cat    461185 non-null float64
dtypes: float64(2), int64(1), uint32(2)
memory usage: 18.2 MB


## Расчет матриц для просмотров и картэдов

In [14]:
#Для просмотров
shape = (user_mapping['new'].max() + 1, item_mapping['new'].max() + 1) # размер матрицы
user_item_view = sp.csr_matrix(arg1=(np.ones_like(user_actions_view['clientid'].values), 
                                       (user_actions_view['clientid'].values, user_actions_view['itemid'].values)), shape=shape)
# all_pairs_knn расчитывает приближенную матрицу похожестей(по N ближайшим соседям)
N = 201
user_similarity_matrix_view = all_pairs_knn(normalize(user_item_view, axis=1), N, show_progress=True, num_threads=2).tocsr()

In [15]:
user_similarity_matrix_view.setdiag(0.)
user_similarity_matrix_view.eliminate_zeros()

/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [16]:
user_similarity_matrix_view.shape

(1052719, 1052719)

In [17]:
#Для картэдов
shape = (user_mapping_tocart['new'].max() + 1, item_mapping_tocart['new'].max() + 1) # размер матрицы
user_item_tocart = sp.csr_matrix(arg1=(np.ones_like(user_actions_tocart['clientid'].values), 
                                       (user_actions_tocart['clientid'].values, user_actions_tocart['itemid'].values)), shape=shape)
    # all_pairs_knn расчитывает приближенную матрицу похожестей(по N ближайшим соседям)
N = 201
user_similarity_matrix_tocart = all_pairs_knn(normalize(user_item_tocart, axis=1), N, show_progress=True, num_threads=2).tocsr()

In [18]:
user_similarity_matrix_tocart.setdiag(0.)
user_similarity_matrix_tocart.eliminate_zeros()

/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [19]:
user_similarity_matrix_tocart.shape

(636764, 636764)

### Расчет похожести рекомендованного товара по  юзер бейсд подходу

In [0]:
def score_used_based(data,user_similarity_matrix,user_item_view, name_score):
  pairs = data.dropna()[['user_cat', 'jointitem_cat']]
  pairs['user_cat'] = pairs['user_cat'].astype('uint32')
  pairs['jointitem_cat'] = pairs['jointitem_cat'].astype('uint32')
  pairs[name_score] = user_similarity_matrix[pairs['user_cat'].values].multiply(user_item_view.T.tocsr()[pairs['jointitem_cat']]).sum(axis=1)
  data = pd.merge(data, pairs, on=['user_cat', 'jointitem_cat'], how = 'left').drop_duplicates()
  return data
  

## для тренировочной выборки

In [23]:
#табличка для проссмотров
df_train_vie = score_used_based(ib_test_target_view,user_similarity_matrix_view ,user_item_view,'us_based_view')
#табличка для добавлений в корзину
df_train_cart = score_used_based(ib_train_target_tocart,user_similarity_matrix_tocart,user_item_tocart,'us_based_tocart')

df_train_vie = df_train_vie.drop(columns = ['user_cat','jointitem_cat'])
df_train_cart = df_train_cart.drop(columns = ['user_cat','jointitem_cat'])
df_train_vie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 597158 entries, 0 to 760985
Data columns (total 4 columns):
clientid         597158 non-null uint32
itemid           597158 non-null int64
jointitemid      597158 non-null uint32
us_based_view    509239 non-null float64
dtypes: float64(1), int64(1), uint32(2)
memory usage: 18.2 MB


In [24]:
train = pd.merge(train_target,df_train_vie[['clientid','itemid','jointitemid','us_based_view']],how = 'left')
train = pd.merge(train,df_train_cart[['clientid','itemid','jointitemid','us_based_tocart']],how = 'left')
train=train.fillna(0)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1390438 entries, 0 to 1390437
Data columns (total 25 columns):
clientid                          1390438 non-null uint32
itemid                            1390438 non-null int64
jointitemid                       1390438 non-null uint32
label                             1390438 non-null int8
timestamp                         1390438 non-null object
view_cnt                          1390438 non-null float64
to_cart_cnt                       1390438 non-null float64
ctr                               1390438 non-null float64
novelty_cnt                       1390438 non-null int32
to_cart_day_avg_cnt               1390438 non-null float64
view_day_avg_cnt                  1390438 non-null float64
to_cart_Lust_Day_cnt              1390438 non-null float64
last_day_views                    1390438 non-null float64
lastDayToCart/dayAvgToCart_cnt    1390438 non-null float64
lastDayView/dayAvgView_cnt        1390438 non-null float64
same_items_o

## Для тестовой выборки

In [25]:
#табличка для проссмотров
df_test_view = score_used_based(ib_test_target_view,user_similarity_matrix_view ,user_item_view,'us_based_view')
#табличка для добавлений в корзину
df_test_tocart = score_used_based(ib_test_target_tocart,user_similarity_matrix_tocart,user_item_tocart,'us_based_tocart')

df_test_view = df_test_view.drop(columns = ['user_cat','jointitem_cat'])
df_test_tocart = df_test_tocart.drop(columns = ['user_cat','jointitem_cat'])

#объеденяем наши таблички, удаляем ненужные столбцы и заполняем пустые значения нулями
test_scores = pd.merge(test_target, df_test_view[['clientid','itemid','jointitemid','us_based_view']],how = 'left')

test_scores = pd.merge(test_scores,df_test_tocart[['clientid','itemid','jointitemid','us_based_tocart']],how = 'left')
test_scores = test_scores.fillna(0)
test_scores.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 597158 entries, 0 to 597157
Data columns (total 25 columns):
clientid                          597158 non-null uint32
itemid                            597158 non-null int64
jointitemid                       597158 non-null uint32
label                             597158 non-null int8
timestamp                         597158 non-null object
view_cnt                          597158 non-null float64
to_cart_cnt                       597158 non-null float64
ctr                               597158 non-null float64
novelty_cnt                       597158 non-null int32
to_cart_day_avg_cnt               597158 non-null float64
view_day_avg_cnt                  597158 non-null float64
to_cart_Lust_Day_cnt              597158 non-null float64
last_day_views                    597158 non-null float64
lastDayToCart/dayAvgToCart_cnt    597158 non-null float64
lastDayView/dayAvgView_cnt        597158 non-null float64
same_items_on_session_view   

## Запись в тестовый и тренеровочный файл

In [0]:
test_scores.to_csv("/content/gdrive/My Drive/python/data/test_with_scores.csv", index = False)
train.to_csv("/content/gdrive/My Drive/python/data/train_with_scores.csv", index = False)